# Feature Engineering for Online Retail II

This notebook converts the raw Online Retail II transactions into customer-level features ready for unsupervised clustering. The focus is on deterministic cleaning and feature creation that feed downstream dimensionality reduction and clustering models.

**Objectives**
- Apply the definitive data quality rules identified during EDA.
- Aggregate transactions to the customer level with meaningful behavioral signals.
- Scale the resulting feature matrix and persist the fitted scaler for reuse.

**Pipeline overview**
1. Load the raw dataset and prepare the working DataFrame.
2. Apply irreversible cleaning rules (missing IDs, cancellations, invalid values, duplicates).
3. Engineer RFM metrics, geographic signals, and basket-based descriptors per customer.
4. Standardize the feature matrix for clustering workflows.
5. Export the processed dataset and scaler artifact for subsequent notebooks.

## 2. Data loading

I read the raw CSV, align column names with the EDA notebook, and create a working copy for feature engineering.

In [1]:
from pathlib import Path
import pickle

import pandas as pd
from sklearn.preprocessing import StandardScaler

DATA_PATH = Path('../data/raw/online_retail_II.csv')
MODELS_DIR = Path('../models')
PROCESSED_DIR = Path('../data/processed')
MODELS_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(DATA_PATH)
df = df.rename(columns={
    'Price': 'UnitPrice',
    'Customer ID': 'CustomerID'
})
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

print(f'Raw shape: {df.shape}\n')
df.info()

df_fe = df.copy()

Raw shape: (1067371, 9)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 9 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1067371 non-null  object        
 1   StockCode    1067371 non-null  object        
 2   Description  1062989 non-null  object        
 3   Quantity     1067371 non-null  int64         
 4   InvoiceDate  1067371 non-null  datetime64[ns]
 5   UnitPrice    1067371 non-null  float64       
 6   CustomerID   824364 non-null   float64       
 7   Country      1067371 non-null  object        
 8   TotalPrice   1067371 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 73.3+ MB


## 3. Data cleaning

Apply the irreversible data quality fixes required before aggregation. All changes below mutate `df_fe` permanently.

### 3.1 Remove rows with missing `CustomerID`

In [2]:
before = df_fe.shape[0]
df_fe = df_fe.dropna(subset=['CustomerID'])
after = df_fe.shape[0]
print(f'Removed {before - after} rows without CustomerID. Remaining: {after}')

Removed 243007 rows without CustomerID. Remaining: 824364


### 3.2 Remove cancelled invoices

In [3]:
cancel_mask = df_fe['Invoice'].astype(str).str.startswith('C', na=False)
removed = cancel_mask.sum()
df_fe = df_fe.loc[~cancel_mask].copy()
print(f'Removed {removed} cancelled transactions. Remaining: {len(df_fe)}')

Removed 18744 cancelled transactions. Remaining: 805620


### 3.3 Remove negative quantities or prices

In [4]:
valid_mask = (df_fe['Quantity'] > 0) & (df_fe['UnitPrice'] > 0)
removed = len(df_fe) - valid_mask.sum()
df_fe = df_fe.loc[valid_mask].copy()
print(f'Removed {removed} rows with non-positive Quantity or UnitPrice. Remaining: {len(df_fe)}')

Removed 71 rows with non-positive Quantity or UnitPrice. Remaining: 805549


### 3.4 Drop exact duplicates

In [5]:
before = len(df_fe)
df_fe = df_fe.drop_duplicates().copy()
after = len(df_fe)
print(f'Removed {before - after} duplicate rows. Remaining: {after}')

Removed 26124 duplicate rows. Remaining: 779425


### 3.5 Optionally drop rows without description

In [6]:
if df_fe['Description'].isna().any():
    before = len(df_fe)
    df_fe = df_fe.dropna(subset=['Description']).copy()
    after = len(df_fe)
    print(f'Removed {before - after} rows with missing Description. Remaining: {after}')
else:
    print('No missing Description values detected')

No missing Description values detected


## 4. Feature engineering

Aggregate the cleaned transactional table at the customer level to obtain RFM metrics (Recency, Frequency, Monetary), geographic footprints, and basket-based descriptors.

### 4.1 RFM metrics

In this step, we transform the data from a transaction-level view (where each row is a single product purchase) to a customer-level view. By grouping by CustomerID and aggregating the metrics, we condense hundreds of thousands of transactions into a single summary row for each unique customer. This explains the significant drop in the number of rows (from ~800k transactions to ~5.8k unique customers), as we are now preparing the data for customer-centric clustering.

In [7]:
snapshot_date = df_fe['InvoiceDate'].max() + pd.Timedelta(days=1)
rfm = (
    df_fe.groupby('CustomerID')
    .agg({
        'InvoiceDate': 'max',
        'Invoice': 'nunique',
        'TotalPrice': 'sum'
    })
    .rename(columns={
        'InvoiceDate': 'LastPurchaseDate',
        'Invoice': 'Frequency',
        'TotalPrice': 'Monetary'
    })
)
rfm['Recency'] = (snapshot_date - rfm['LastPurchaseDate']).dt.days
rfm = rfm.drop(columns=['LastPurchaseDate'])
customer_features = rfm.copy()
customer_features.head()

,Frequency,Monetary,Recency
CustomerID,,,
12346.0,12,77556.46,326
12347.0,8,4921.53,2
12348.0,5,2019.40,75
12349.0,4,4428.69,19
12350.0,1,334.40,310


### 4.2 Country one-hot encoding

In [8]:
def most_frequent(series: pd.Series) -> str:
    mode = series.mode()
    return mode.iloc[0] if not mode.empty else series.iloc[0]

customer_country = df_fe.groupby('CustomerID')['Country'].agg(most_frequent)
country_dummies = pd.get_dummies(customer_country, prefix='Country')
customer_features = customer_features.join(country_dummies, how='left')
print(f'Country dummy shape: {country_dummies.shape}\n')
customer_features.head()

Country dummy shape: (5878, 41)



,Frequency,Monetary,Recency,Country_Australia,Country_Austria,Country_Bahrain,Country_Belgium,Country_Brazil,Country_Canada,Country_Channel Islands,...,Country_Singapore,Country_Spain,Country_Sweden,Country_Switzerland,Country_Thailand,Country_USA,Country_United Arab Emirates,Country_United Kingdom,Country_Unspecified,Country_West Indies
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,12,77556.46,326,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
12347.0,8,4921.53,2,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
12348.0,5,2019.40,75,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
12349.0,4,4428.69,19,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
12350.0,1,334.40,310,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### 4.3 Basket statistics and product diversity

In [9]:
invoice_summary = (
    df_fe.groupby(['CustomerID', 'Invoice'])
    .agg(InvoiceRevenue=('TotalPrice', 'sum'), InvoiceQuantity=('Quantity', 'sum'))
    .reset_index()
)
avg_basket = invoice_summary.groupby('CustomerID')['InvoiceRevenue'].mean().rename('AvgBasketValue')
var_basket = invoice_summary.groupby('CustomerID')['InvoiceRevenue'].var(ddof=0).rename('BasketValueVariance')
distinct_products = df_fe.groupby('CustomerID')['StockCode'].nunique().rename('DistinctProducts')
basket_features = pd.concat([avg_basket, var_basket, distinct_products], axis=1)
basket_features = basket_features.fillna({'BasketValueVariance': 0, 'AvgBasketValue': 0, 'DistinctProducts': 0})
customer_features = customer_features.join(basket_features, how='left')
customer_features[['AvgBasketValue', 'BasketValueVariance', 'DistinctProducts']] = (
    customer_features[['AvgBasketValue', 'BasketValueVariance', 'DistinctProducts']].fillna(0)
)
customer_features = customer_features.sort_index()
df_model = customer_features.reset_index().rename(columns={'index': 'CustomerID'})
print(f'Customer-level feature matrix shape: {df_model.shape}')

Customer-level feature matrix shape: (5878, 48)


## 5. Scaling

Standardize the customer-level features (excluding the identifier) to make them comparable for clustering.

In [10]:
feature_columns = [col for col in df_model.columns if col != 'CustomerID']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_model[feature_columns])
df_model_scaled = pd.DataFrame(X_scaled, columns=feature_columns, index=df_model.index)
df_model_scaled.insert(0, 'CustomerID', df_model['CustomerID'].values)
display(df_model_scaled.head())

,CustomerID,Frequency,Monetary,Recency,Country_Australia,Country_Austria,Country_Bahrain,Country_Belgium,Country_Brazil,Country_Canada,...,Country_Switzerland,Country_Thailand,Country_USA,Country_United Arab Emirates,Country_United Kingdom,Country_Unspecified,Country_West Indies,AvgBasketValue,BasketValueVariance,DistinctProducts
0,12346.0,0.438998,5.166378,0.595584,-0.048862,-0.0433,-0.018449,-0.069183,-0.018449,-0.029178,...,-0.061293,-0.013044,-0.03916,-0.026095,0.314152,-0.031966,-0.013044,5.005717,4.882138,-0.472112
1,12347.0,0.131502,0.136127,-0.952279,-0.048862,-0.0433,-0.018449,-0.069183,-0.018449,-0.029178,...,-0.061293,-0.013044,-0.03916,-0.026095,-3.183171,-0.031966,-0.013044,0.189436,-0.014699,0.377858
2,12348.0,-0.099120,-0.064857,-0.603532,-0.048862,-0.0433,-0.018449,-0.069183,-0.018449,-0.029178,...,-0.061293,-0.013044,-0.03916,-0.026095,-3.183171,-0.031966,-0.013044,0.015401,-0.014964,-0.489283
3,12349.0,-0.175994,0.101996,-0.871064,-0.048862,-0.0433,-0.018449,-0.069183,-0.018449,-0.029178,...,-0.061293,-0.013044,-0.03916,-0.026095,-3.183171,-0.031966,-0.013044,0.594632,-0.012044,0.480885
4,12350.0,-0.406616,-0.181549,0.519146,-0.048862,-0.0433,-0.018449,-0.069183,-0.018449,-0.029178,...,-0.061293,-0.013044,-0.03916,-0.026095,-3.183171,-0.031966,-0.013044,-0.041823,-0.015639,-0.557968


## 6. Export

Persist the scaled dataset for PCA/clustering workflows along with the fitted scaler.

In [11]:
processed_path = PROCESSED_DIR / 'customers_final.csv'
scaler_path = MODELS_DIR / 'scaler.pkl'
df_model_scaled.to_csv(processed_path, index=False)
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)
print(f'Saved processed dataset to {processed_path}')
print(f'Saved scaler to {scaler_path}')

Saved processed dataset to ..\data\processed\customers_final.csv
Saved scaler to ..\models\scaler.pkl
